In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/made-hw-2/test.csv
/kaggle/input/made-hw-2/train.csv
/kaggle/input/made-hw-2/sample_submission_most_popular.csv


In [7]:
train = pd.read_csv('/kaggle/input/made-hw-2/train.csv')
test = pd.read_csv('/kaggle/input/made-hw-2/test.csv')
sub = pd.read_csv('/kaggle/input/made-hw-2/sample_submission_most_popular.csv')

In [8]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + \
                    ' '.join(emoticons).replace('-', '')
    return text

In [9]:
train['dialogue'] = train['dialogue'].apply(preprocessor)

In [10]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train['dialogue'], train['genres'], 
                                                      test_size=0.1, random_state=42)

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [21]:
sgd = Pipeline([('tfidf', TfidfVectorizer()),
               ('sgd_clf', SGDClassifier(random_state=42))])
knb = Pipeline([('tfidf', TfidfVectorizer()),
               ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd.fit(x_train, y_train)
knb.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

In [35]:
sub

,id,genres
0,0,drama
1,1,drama
2,2,drama
3,3,drama
4,4,drama
...,...,...
9398,9398,drama
9399,9399,drama
9400,9400,drama
9401,9401,drama


In [36]:
x_test = train.loc[:9402, 'dialogue'].apply(preprocessor)
predicted_sgd = sgd.predict(x_test)
y_test = sub['genres']

In [38]:
print(metrics.classification_report(predicted_sgd, y_test))

ValueError: Found input variables with inconsistent numbers of samples: [9404, 9403]